In [9]:
import plotly.express as px
import matplotlib.pyplot as plt
import panel as pn
import pandas as pd
import os
from pathlib import Path
from dotenv import load_dotenv
# Set up Panel Plotly extension
pn.extension('plotly')
# Import hvplot.pandas after pn.extension
import hvplot.pandas


In [10]:
# Load .env
load_dotenv()
mapbox_api_key = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(mapbox_api_key)

# Reading GTA Neighbourhood data
people_path = Path("Data/toronto_neighbourhoods_historical_people_data.csv")
people_df= pd.read_csv(people_path, index_col="neighbourhood").dropna()
people_df.head()


,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
neighbourhood,,,,,,,,,,,,,,,,,,,,,
Agincourt North,3,1,1,1,6,2,6,7,7,5,...,2,6,6,2,1,2,3,4,4,4
Agincourt South-Malvern West,1,2,4,3,5,3,6,8,10,7,...,9,7,10,10,10,10,7,10,10,7
Alderwood,1,2,1,1,1,1,3,1,2,6,...,6,6,6,7,10,6,6,3,1,1
Annex,1,6,10,10,10,10,10,6,2,7,...,5,3,3,1,1,5,8,8,10,10
Banbury-Don Mills,4,7,4,8,8,7,8,10,6,10,...,8,7,10,10,9,10,7,4,7,8


In [11]:
# Calculating growth rates
growth = people_df.pct_change(axis=1).dropna(axis=1)

# Calculating the average growth
average_growth = growth.mean(axis=1)

# Sort by the most growth rate
average_growth = average_growth.sort_values(ascending=False)

average_growth = average_growth.reset_index()
average_growth.columns = ['neighbourhood', 'Average Growth Rate']
average_growth.head()

,neighbourhood,Average Growth Rate
0,Pleasant View,0.777321
1,Victoria Village,0.674881
2,Beechborough-Greenbrook,0.653889
3,Caledonia-Fairbank,0.605000
4,Dovercourt-Wallace Emerson-Juncti,0.573155


In [12]:

# Plotting the growth
average_growth.hvplot.bar(x="neighbourhood", xlabel='Neighbourhood', ylabel='Avg. Growth Rate', title="GTA Neighbourhoods by the most growth rate", height=700, width= 1900, rot=90, yformatter='%.0f')

:Bars   [neighbourhood]   (Average Growth Rate)

In [13]:
# Loading Coordinations
geo_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
geo_df = pd.read_csv(geo_path).dropna()
geo_df.head()


,neighbourhood,lat,lon
0,Agincourt North,43.805441,-79.266712
1,Agincourt South-Malvern West,43.788658,-79.265612
2,Alderwood,43.604937,-79.541611
3,Annex,43.671585,-79.404001
4,Banbury-Don Mills,43.737657,-79.349718


In [14]:
# Combining datasets
neighbourhood_growth_locations = pd.concat([geo_df, average_growth.drop(columns=['neighbourhood'])], axis="columns", join="inner")
neighbourhood_growth_locations.head()

,neighbourhood,lat,lon,Average Growth Rate
0,Agincourt North,43.805441,-79.266712,0.777321
1,Agincourt South-Malvern West,43.788658,-79.265612,0.674881
2,Alderwood,43.604937,-79.541611,0.653889
3,Annex,43.671585,-79.404001,0.605000
4,Banbury-Don Mills,43.737657,-79.349718,0.573155


In [15]:
# Plotting the map
def neighbourhood_growth_map():
    """Neighbourhood Growth Map"""  
    return px.scatter_mapbox(
        neighbourhood_growth_locations,
        lat="lat",
        lon="lon",
        size="Average Growth Rate",
        color="Average Growth Rate",
        zoom=9,
        width=1000,
        height=600,
        title="Growth Rates in Toronto",
        color_continuous_scale=px.colors.cyclical.IceFire
    )

neighbourhood_growth_map_column = pn.Column("# Neighbourhood Growth Map", neighbourhood_growth_map())

# Dashboard Tab
dashboard = pn.Tabs(    
    (
        "Map",
        neighbourhood_growth_map_column
    )
)

# Create the main dashboard
dashboard

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)